In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_neo4j import Neo4jGraph

#LangChain 도구 활용 - DB연결 객체 초기화
graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD"),
)

In [5]:
# 테스트 쿼리 실행
cypher_query="""
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

### 2. 전문 검색 (Full-Text Search)활용

In [6]:
#Movie 노드의 title 속성에 대한 전문 검색 인텍스 생성
#CREATE FULLTEXT INDEX : 전문 검색 (Full-text search)을 위한 인덱스를 생성하는 Cypher 명령어
#movie_title_fulltext : 생성할 인덱스의 이름 (나중에 이 이름으로 인덱스를 참조)
#FOR (m:Movie) : Movie 라벨을 가진 모든 노드에 대해 인덱스 적용
#ON EACH [m.title] : 각 Movie 노드의 title 속성만 인덱싱 (여러 속성을 인덱싱 하려면 배열에 추가 가능)
fulltext_index_query="""
CREATE FULLTEXT INDEX movie_title_fulltext IF NOT EXISTS
FOR (m:Movie) ON EACH [m.title]
"""

graph.query(fulltext_index_query)

[]

In [7]:
graph.query("SHOW FULLTEXT INDEXES")

[{'id': 8,
  'name': 'movie_title_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

In [12]:
#Movie 노드의 title과 tagline 속성 모두에 대한 전문 검색 인덱스 생성

fulltext_index_query="""
CREATE FULLTEXT INDEX movie_title_tagline IF NOT EXISTS
FOR (m:Movie) ON EACH [m.title, m.tagline]
"""

graph.query(fulltext_index_query)

[]

In [13]:
graph.query("SHOW FULLTEXT INDEXES")

[{'id': 8,
  'name': 'movie_title_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 9,
  'name': 'movie_title_tagline',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title', 'tagline'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': None}]

In [14]:
#Person 노드의 name 속성에 전문 검색 인덱스 생성
person_fulltext_index_query="""
CREATE FULLTEXT INDEX person_name_fulltext IF NOT EXISTS
FOR (p:Person) ON EACH [p.name]
"""
graph.query(person_fulltext_index_query)

[]

In [15]:
graph.query("SHOW FULLTEXT INDEXES")

[{'id': 8,
  'name': 'movie_title_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 9,
  'name': 'movie_title_tagline',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title', 'tagline'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 10,
  'name': 'person_name_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Person'],
  'properties': ['name'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

### 2.2 검색쿼리

1. 점수의 의미: BM25 알고리즘
Neo4j는 내부적으로 Apache Lucene을 사용하며, 기본적으로 "BM25(Best Matching 25)"라는 알고리즘을 통해 점수를 산출합니다.

숫자가 높을수록: 입력하신 검색어($search_term)와 해당 노드의 속성(name)이 더 밀접하게 관련되어 있다는 뜻입니다.

상대적 수치: 이 점수는 0에서 1 사이의 고정된 값이 아닙니다. 검색어의 희귀성, 문서의 길이 등에 따라 10점, 100점이 될 수도 있는 상대적인 점수입니다.

2. 점수를 결정하는 주요 요소
SearchRelevance 값은 크게 세 가지 요소에 의해 결정됩니다.

단어 빈도 (TF, Term Frequency): 노드 내에 검색어(예: "Tom")가 얼마나 자주 등장하는가? (다만 이름 필드에서는 보통 한 번만 등장하므로 영향이 적습니다.)

역문서 빈도 (IDF, Inverse Document Frequency): "Tom"이라는 단어가 전체 DB 노드들 사이에서 얼마나 희귀한가? 만약 "Tom"이라는 이름이 아주 흔하다면 점수가 낮아지고, "Zendaya"처럼 희귀하다면 점수가 높아집니다.

필드 길이 (Field Length): 이름 전체 길이 중 검색어가 차지하는 비중입니다. 예를 들어 "Tom"을 검색했을 때:

이름이 딱 **"Tom"**인 노드 (점수 높음 ⬆️)

이름이 **"Tom Cruise"**인 노드 (점수 중간 ⬇️)

이름이 **"Thomas 'Tom' Steyer"**인 노드 (점수 낮음 ⬇️)

In [16]:
# 'love'라는 단어가 포함된 영화 제목을 검색
# 전문 검색 인덱스를 사용하여 영화 제목에서 'love' 키워드 검색
search_query="""
//db.index.fulltext.queryNodes : 전문 검색 인덱스를 사용하여 노드를 검색하는 프로시저
//"movie_title_fulltext" : 앞서 생성한 영화제목 전문 검색 인덱스 이름
// $search_term : 파라미터로 전달되는 검색어 (여기서는 'love')

CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)

//YIELD : 프로시저의 결과를 반환
//node : 검색된 노드 객체, score : 검색 관련성 점수

YIELD node, score

RETURN node.title AS MovieTitle, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

graph.query(search_query, params={"search_term": "love"})

[{'MovieTitle': 'Love Stinks', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love & Basketball', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love Happens', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love Ranch', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love Letters', 'SearchRelevance': 2.281766891479492}]

In [18]:
#배우/감독 이름 검색
search_query="""
CALL db.index.fulltext.queryNodes("person_name_fulltext", $search_term)
YIELD node, score
RETURN node.name AS PersonName, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 3
"""

graph.query(search_query, params={"search_term":"tom"})

[{'PersonName': 'Tom Noonan', 'SearchRelevance': 2.360572099685669},
 {'PersonName': 'Tom Hulce', 'SearchRelevance': 2.360572099685669},
 {'PersonName': 'Tom Bosley', 'SearchRelevance': 2.360572099685669}]

In [19]:
#퍼지검색 - 오타를 허용하는 검색 (예: 'afair'로 'affair' 찾기)
fuzzy_search_query="""
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score
RETURN node.title AS MovieTitle, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

#~0.7은 편집 거리 (edit distance)를 기반으로 70% 유사도를 가진 결과까지 포함
fuzzy_results = graph.query(fuzzy_search_query, params={"search_term": "afair~0.7"})

for result in fuzzy_results:
    print(f"{result['MovieTitle']} (관련도: {result['SearchRelevance']})")

Vanity Fair (관련도: 2.685884475708008)
Fair Game (관련도: 2.685884475708008)
State Fair (관련도: 2.685884475708008)
The Thomas Crown Affair (관련도: 2.3315298557281494)
My Fair Lady (관련도: 2.3031468391418457)


In [20]:
# 와일드카드 검색
wildcard_search_query="""
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score
RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

#star* : 'star'로 시작하는 모든 단어를 검색
wildcard_results = graph.query(wildcard_search_query, params={"search_term":"star*"})

print("\n=== 와일드카드 검색 결과(star*)===")
for result in wildcard_results:
    print(f"{result['MovieTitle']}({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


=== 와일드카드 검색 결과(star*)===
Star Trek Beyond(2016-07-07) - 관련도: 1.0000
The Fault in Our Stars(2014-05-16) - 관련도: 1.0000
My Lucky Star(2013-09-17) - 관련도: 1.0000
20 Feet from Stardom(2013-06-14) - 관련도: 1.0000
Star Wars(1977-05-25) - 관련도: 1.0000


In [ ]:
#정확한 구문 검색 - 정확한 문구를 검색 (예: 'love story')
phrase_search_query="""
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score
RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

#이스케이프 처리된 쌍따옴표를 사용하여 정확한 구문을 지정
phrase_results = graph.query(phrase_search_query, params={"search_term":"\"love story\""})

print("\n===정확한 구문 검색 결과 (\"love story\") ===")
for result in phrase_results:
    print(f"{result['MovieTitle']} ({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


===정확한 구문 검색 결과 ("love story") ===
Wristcutters: A Love Story (2006-01-24) - 관련도: 3.6628
Capitalism: A Love Story (2009-09-06) - 관련도: 3.6628


In [24]:
# 논리 연산자 검색 - 특정 단어를 포함하고 다른 단어는 제외 (예: 'love'와 'story'를 포함하고 'horror'는 제외)
boolean_search_query="""
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score
RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

#love AND story NOT horror : 'love'와 'story'를 포함하고 'horror'는 제외하는 검색
boolean_results = graph.query(boolean_search_query, params={"search_term": "love AND story NOT horror"})

print("\n===논리 연산자 검색 결과 (love AND story NOT horror) ====")
for result in boolean_results:
    print(f"{result['MovieTitle']} ({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


===논리 연산자 검색 결과 (love AND story NOT horror) ====
Wristcutters: A Love Story (2006-01-24) - 관련도: 3.6628
Capitalism: A Love Story (2009-09-06) - 관련도: 3.6628


In [25]:
#가중치 부여 검색 - 특정 단어에 더 높은 가중치 부여 (예: 'love'에 4배 가중치)
boost_search_query="""
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score
RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

#love^4 story : 'love'에 4배 가중치를 부여한 검색
boost_results = graph.query(boost_search_query, params={"search_term": "love^4 story"})

print("\n===가중치 부여 검색 결과 (love^4 story) ====")
for result in boost_results:
    print(f"{result['MovieTitle']} ({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


===가중치 부여 검색 결과 (love^4 story) ====
Love Jones (1997-03-14) - 관련도: 9.1271
Love Actually (2003-09-07) - 관련도: 9.1271
Love Ranch (2010-06-30) - 관련도: 9.1271
Endless Love (2014-02-12) - 관련도: 9.1271
Love Letters (1983-04-01) - 관련도: 9.1271


### 2.3 전문 검색과 그래프 탐색 결합

In [26]:
#전문 검색 -> 그래프 탐색 : 특정 단어를 포함하는 영화를 먼저 찾고 (full-text), 각 영화에 출연한 출연배우 찾기 (graph)

combined_search_query="""
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node as movie, score
MATCH (movie)<-[:ACTED_IN]-(actor:Person)
RETURN movie.title AS MovieTitle, score AS SearchRelevance, collect(actor.name) AS Actors
ORDER BY SearchRelevance DESC
LIMIT 5
"""

combined_results = graph.query(combined_search_query, params={"search_term": "love"})

for result in combined_results:
    print(f"{result['MovieTitle']} (관련도: {result['SearchRelevance']})")
    print(f"   출연배우:{', '.join(result['Actors'])}")
    print()

Love Stinks (관련도: 2.281766891479492)
   출연배우:Bridgette Wilson, Bill Bellamy, Tiffani Thiessen, French Stewart, Tyra Banks

Love & Basketball (관련도: 2.281766891479492)
   출연배우:Alfre Woodard, Omar Epps, Sanaa Lathan, Chris Warren, Jr., Kyla Pratt

Love Happens (관련도: 2.281766891479492)
   출연배우:Martin Sheen, Jennifer Aniston, Aaron Eckhart, Judy Greer, Deirdre Blades

Love Ranch (관련도: 2.281766891479492)
   출연배우:Joe Pesci, Bai Ling, Gina Gershon, Helen Mirren, Sergio Peris-Mencheta

Love Letters (관련도: 2.281766891479492)
   출연배우:Jamie Lee Curtis, James Keach, Matt Clark, Bonnie Bartlett, Amy Madigan

